Очистка данных

Цели:
* привести данные к корректным типам;
* обработать пропуски;
* найти и обработать возвраты;
* удалить дубли;
* подготовить чистый датасет для дальнейшего анализа

In [36]:
import pandas as pd
import numpy as np
df = pd.read_csv('../data/online_retail.csv', encoding='ISO-8859-1')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [37]:
# приведем дату покупки к datetime (т.к. сейчас это строка)
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate'].dtype

dtype('<M8[ns]')

In [38]:
# id покупателей к nullable integer
df['CustomerID'] = df['CustomerID'].astype('Int64')

ид покупателя - ключ, его формат был float, т.к. обычный int не может хранить пропуски (NaN), а они тут есть. 
int64 хранит целые числа + пропуски.

Такой тип данных более корректен

In [39]:
# повторяю % соотношение пропусков
(df.isna().sum() / len(df) * 100).round(5)

InvoiceNo       0.00000
StockCode       0.00000
Description     0.26831
Quantity        0.00000
InvoiceDate     0.00000
UnitPrice       0.00000
CustomerID     24.92669
Country         0.00000
dtype: float64

Название товара - пропуски менее 1%, можно удалить

In [40]:
# удаляем строки, где пустое название товара
df['Description'].isna().sum()
df = df.dropna(subset=['Description'])

In [41]:
df.duplicated().sum()

np.int64(5268)

In [42]:
#удаляем дубли
df = df.drop_duplicates()

In [43]:
# отдельный датасет для возвратов:
returns = df[df['Quantity'] < 0]

In [44]:
# размерность (сколько возвратов):
returns.shape

(9725, 8)

In [45]:
# очищенный от возвратов датасет:
df_clean = df[df['Quantity'] > 0].copy()

In [46]:
# создаем столбец с выручкой:
df_clean['Revenue'] = df_clean['Quantity'] * df_clean['UnitPrice']

In [47]:
df_clean['Revenue'].describe()

count    525462.000000
mean         20.210761
std         272.402776
min      -11062.060000
25%           3.900000
50%           9.920000
75%          17.700000
max      168469.600000
Name: Revenue, dtype: float64

Но удалены отрицательные количества (возвраты), значит есть отрицательная цена. Исправляем:

In [48]:
df_clean[df_clean['UnitPrice'] <= 0].shape

(584, 9)

In [49]:
df_clean = df_clean[df_clean['UnitPrice'] > 0]

In [50]:
#проверка:
df_clean['Revenue'].describe()

count    524878.000000
mean         20.275399
std         271.693566
min           0.001000
25%           3.900000
50%           9.920000
75%          17.700000
max      168469.600000
Name: Revenue, dtype: float64

In [51]:
df_clean['Quantity'].quantile([0.01, 0.99])

0.01      1.0
0.99    100.0
Name: Quantity, dtype: float64

99% заказов имеют не более 100шт товаров в заказке. Больше 100 позиций - аномалия или просто крупные покупки? 

In [52]:
# проверка очищеного датасета:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 524878 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    524878 non-null  object        
 1   StockCode    524878 non-null  object        
 2   Description  524878 non-null  object        
 3   Quantity     524878 non-null  int64         
 4   InvoiceDate  524878 non-null  datetime64[ns]
 5   UnitPrice    524878 non-null  float64       
 6   CustomerID   392692 non-null  Int64         
 7   Country      524878 non-null  object        
 8   Revenue      524878 non-null  float64       
dtypes: Int64(1), datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 40.5+ MB


In [53]:
df_clean.isna().sum()

InvoiceNo           0
StockCode           0
Description         0
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     132186
Country             0
Revenue             0
dtype: int64

In [54]:
# сохраняем очищенный датасет:
df_clean.to_csv("../data/online_retail_clean.csv", index=False)

### Итог по очистке данных:
* приведение к более корректным типам данных;
* обработаны пропуски;
* возвраты в отдельном датасете;
* создан столбец с выручкой (цена*количество);
Создан очищенный датасет для анализа!